<a href="https://colab.research.google.com/github/M-torki/ECG-Classification/blob/main/Sprint5_Automatic_Diagnosis_mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd /gdrive/MyDrive/

/gdrive/MyDrive


In [ ]:
!pip install mlflow

     |████████████████████████████████| 14.4 MB 61 kB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 170 kB 54.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 146 kB 54.5 MB/s 
     |████████████████████████████████| 636 kB 52.1 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
     |████████████████████████████████| 68 kB 5.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158170 sha256=2d12eb3083d53c6cd00b93010bf3056b5c7e9b87e239120ce1d3fd7dd34be4e8
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for databricks-cli: filename=databricks_cli-0.15.0-py3-none-any.whl size=105259 sha256=1012ae0ae8b384cdd1dd0714a46b40f36c2b1e7417e9ff813027d24d4545f79f

In [ ]:
!pip install wfdb

     |████████████████████████████████| 137 kB 6.1 MB/s 


In [ ]:
from wfdb import plot_items
from wfdb.processing import resample_sig , find_peaks , resample_singlechan
import mlflow
import mlflow.keras
import mlflow.tensorflow

#Data Preparation

#utils:

In [ ]:
#@title
import scipy
import numpy as np
from glob import glob
import pywt
from wfdb.processing import resample_sig
from scipy.signal import kaiserord, firwin, filtfilt, butter

In [ ]:
#@title
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def wavelet_denoising(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    # print(madev(coeff[-level]))
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    # print('sigma' , sigma ,'uthresh' , uthresh)
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='soft') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


def FIRRemoveBL(ecgy, Fs, Fc, factor):
    
    #    ecgy:        the contamined signal (must be a list)
    #    Fc:          cut-off frequency
    #    Fs:          sample frequiency
    #    ECG_Clean :  processed signal without BLW
    
    # getting the length of the signal
    signal_len = len(ecgy)
    
    # The Nyquist rate of the signal.
    nyq_rate = Fs / 2.0
    
    # The desired width of the transition from stop to pass,
    # relative to the Nyquist rate. 
    width = 0.07/nyq_rate 
    
    # Attenuation in the stop band, in dB.
    # related to devs in Matlab. On Matlab is on proportion
    ripple_db = round(-20*np.log10(0.001))+1
    ripple_db = ripple_db / factor

    
    # Compute the order and Kaiser parameter for the FIR filter.
    N, beta = kaiserord(ripple_db, width)
    N |= 1
    # Use firwin with a Kaiser window to create a highpass FIR filter.
    h = firwin(N, Fc/nyq_rate, window=('kaiser', beta), pass_zero='highpass')

    # Check filtfilt condition
    if N*3 > signal_len:
        diff = N*3 - signal_len
        ecgy = list(reversed(ecgy)) + list(ecgy) + list(ecgy[-1] * np.ones(diff))
        
        # Filtering with filtfilt
        ECG_Clean = filtfilt(h, 1.0, ecgy)
        ECG_Clean = ECG_Clean[signal_len: signal_len + signal_len]
    else:
        ECG_Clean = filtfilt(h, 1.0, ecgy)
    
    return ECG_Clean, N


def FIRRemoveHF(ecgy, Fs, Fc, factor):
    #    ecgy:        the contamined signal (must be a list)
    #    Fc:          cut-off frequency
    #    Fs:          sample frequiency
    #    ECG_Clean :  processed signal without BLW

    # getting the length of the signal
    signal_len = len(ecgy)

    # The Nyquist rate of the signal.
    nyq_rate = Fs / 2.0

    # The desired width of the transition from stop to pass,
    # relative to the Nyquist rate.
    width = 0.07 / nyq_rate

    # Attenuation in the stop band, in dB.
    # related to devs in Matlab. On Matlab is on proportion
    ripple_db = round(-20 * np.log10(0.001)) + 1
    ripple_db = ripple_db / factor

    # Compute the order and Kaiser parameter for the FIR filter.
    N, beta = kaiserord(ripple_db, width)
    N |= 1
    # Use firwin with a Kaiser window to create a highpass FIR filter.
    h = firwin(N, Fc / nyq_rate, window=('kaiser', beta), pass_zero='lowpass')

    # Check filtfilt condition
    if N * 3 > signal_len:
        diff = N * 3 - signal_len
        ecgy = list(reversed(ecgy)) + list(ecgy) + list(ecgy[-1] * np.ones(diff))

        # Filtering with filtfilt
        ECG_Clean = filtfilt(h, 1.0, ecgy)
        ECG_Clean = ECG_Clean[signal_len: signal_len + signal_len]
    else:
        ECG_Clean = filtfilt(h, 1.0, ecgy)

    return ECG_Clean, N

In [ ]:
#@title
#################### Constants and Arguments  ##########################

path_to_data_dir = './export/raw/'

leads = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'v3', 'v4', 'v5', 'v6'] 
n_channel = 12
fs_ini = 350
fs_target = 400
n_sample = 4096
down_sample = 1
filename = sorted(glob(str(path_to_data_dir)+'*.txt'))
filename[0]

'./export/raw/1.txt'

In [ ]:
#@title
# from utils import *

class DataProvider(object):
    def __init__(self,data_files,n_class,
                 n_sample=4096,
                 n_channel=12, 
                 fs_ini = 350,
                 fs_target = None,
                 wav = None,
                 wav_level = None,
                 Fc_l = None,
                 down_sample = 1,
                 filtering =1 ):

        self.data_files = data_files
        self.n_channel = n_channel
        self.fs_ini = fs_ini
        self.n_sample = n_sample
        self.down_sample = down_sample
        self.filtering = filtering

        self.fs_target = fs_target

        self.wav = wav
        self.level = wav_level
        self.Fc_l = Fc_l
            # self.Fc_h = Fc_h 


    def pre_process(self , signal):

        if self.down_sample:
            resampled = []
            for j in range(self.n_channel):
                # print(signal.shape)
                s = resample_sig(signal[j,:] , fs=self.fs_ini , fs_target=self.fs_target)[0][:self.n_sample]
                # print(s.shape)
                if s.shape[0] < self.n_sample:
                    a = (self.n_sample - s.shape[0])//2
                    s = np.pad( s,(a,a))
                resampled.append(s)
        else:
            resampled = signal
            self.fs_target = self.fs_ini
        resampled = np.array(resampled)

        if self.filtering:
            fil = []
            for j in range(self.n_channel):
                # print(resampled.shape)
                sa = resampled[j,:]
                # print(sa.shape)
                sa_f , N = FIRRemoveBL(sa , self.fs_target, self.Fc_l , factor=4.5)
                # sa_f, N = FIRRemoveHF(sa_f, self.fs_target, self.Fc_h, 4.5)
                sa_f =  wavelet_denoising(sa_f, wavelet=self.wav, level=self.level)
                fil.append(sa_f)
        else:
            fil = resampled

        fil = np.array(fil)
        # print(fil.shape)

        if fil.shape[-1] != self.n_channel:
            fil = np.transpose(fil)
            fil = np.array(fil)

        return fil


    def __call__(self,n): 

        X = []
        Y = []
        for f in self.data_files[:n]: 
            rec = np.loadtxt(f , skiprows=1 , delimiter=';'  , unpack=True)
            # if rec[0,0] > 100:
            rec = rec/100
            # print(rec.shape)
            rec = self.pre_process(rec)
            # print(rec.shape)
            X.append(rec)
        
        X = np.array(X)
        # Y = np.array(Y)

        return X#,Y

In [ ]:
#@title
dp = DataProvider(data_files=filename, n_class=6 , fs_target=400 ,
                  wav='sym5' , wav_level=1 , Fc_l =0.67,
                  down_sample=1 , filtering=1)
X = dp(5)
print(X.shape)

t = np.arange(0,10,10/4096)
x = np.loadtxt(filename[1] , skiprows=1 , delimiter=';'  , unpack=True)[0,:4096]/100

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
plt.title('Both Downsampling & Filtering' , fontsize=14)
plt.plot(t , x ,'black', label='Raw')
plt.plot(t ,X[1,:,0] , 'lightcoral',label='Filtered')

In [ ]:
#@title
dp = DataProvider(data_files=filename, n_class=6 , fs_target=400 ,
                  down_sample=1 , filtering=0)
X = dp(5)
print(X.shape)

t = np.arange(0,10,10/4096)
x = np.loadtxt(filename[1] , skiprows=1 , delimiter=';'  , unpack=True)[0,:4096]/100

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
plt.title('With Downsampling , without filtering' , fontsize=14)
plt.plot(t , x ,'black', label='Raw')
plt.plot(t ,X[1,:,0] , 'lightcoral',label='Filtered')

In [ ]:
#@title
dp = DataProvider(data_files=filename, n_class=6 , fs_target=400 ,
                  down_sample=0 , filtering=0)
X = dp(5)
print(X.shape)

t = np.arange(0,10,10/6000)
x = np.loadtxt(filename[1] , skiprows=1 , delimiter=';'  , unpack=True)[0,:6000]

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
plt.title('With niether filtering nor downsampling' , fontsize=14)
plt.plot(t , x ,'black', label='Raw')
plt.plot(t ,X[1,:,0] , 'lightcoral',label='Filtered')

In [ ]:
#@title
dp = DataProvider(data_files=filename, n_class=6 ,
                wav='sym5' , wav_level=1 , Fc_l =0.67,
                  down_sample=0 , filtering=1)
X = dp(5)
print(X.shape)

t = np.arange(0,10,10/6000)
x = np.loadtxt(filename[1] , skiprows=1 , delimiter=';'  , unpack=True)[0,:6000]

import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
plt.title('with filtering , without downsampling' , fontsize=14)
plt.plot(t , x ,'black', label='Raw')
plt.plot(t ,X[1,:,0] , 'lightcoral',label='Filtered')

In [ ]:
#@title


#Model definition, training & predictions:

Datasets:

In [ ]:
cd ./ecg_ptbxl_benchmarking/

/gdrive/.shortcut-targets-by-id/1j3ncHY23bba4nXCRAt52Fr8YgGZpml9-/ecg_ptbxl_benchmarking


In [ ]:
cd code/

/gdrive/.shortcut-targets-by-id/1j3ncHY23bba4nXCRAt52Fr8YgGZpml9-/ecg_ptbxl_benchmarking/code


In [ ]:
#@title utils
import os
import sys
import re
import glob
import pickle
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import wfdb
import ast
from sklearn.metrics import classification_report, fbeta_score, roc_auc_score, roc_curve, roc_curve, auc
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from matplotlib.axes._axes import _log as matplotlib_axes_logger
import warnings

# EVALUATION STUFF
def generate_results(idxs, y_true, y_pred, thresholds):
    return evaluate_experiment(y_true[idxs], y_pred[idxs], thresholds)

def evaluate_experiment(y_true, y_pred, thresholds=None):
    results = {}

    if not thresholds is None:
        # binary predictions
        y_pred_binary = apply_thresholds(y_pred, thresholds)
        # PhysioNet/CinC Challenges metrics
        challenge_scores = challenge_metrics(y_true, y_pred_binary, beta1=2, beta2=2)
        results['F_beta_macro'] = challenge_scores['F_beta_macro']
        results['G_beta_macro'] = challenge_scores['G_beta_macro']

    # label based metric
    results['macro_auc'] = roc_auc_score(y_true, y_pred, average='macro')
    
    df_result = pd.DataFrame(results, index=[0])
    return df_result

def challenge_metrics(y_true, y_pred, beta1=2, beta2=2, class_weights=None, single=False):
    f_beta = 0
    g_beta = 0
    if single: # if evaluating single class in case of threshold-optimization
        sample_weights = np.ones(y_true.sum(axis=1).shape)
    else:
        sample_weights = y_true.sum(axis=1)
    for classi in range(y_true.shape[1]):
        y_truei, y_predi = y_true[:,classi], y_pred[:,classi]
        TP, FP, TN, FN = 0.,0.,0.,0.
        for i in range(len(y_predi)):
            sample_weight = sample_weights[i]
            if y_truei[i]==y_predi[i]==1: 
                TP += 1./sample_weight
            if ((y_predi[i]==1) and (y_truei[i]!=y_predi[i])): 
                FP += 1./sample_weight
            if y_truei[i]==y_predi[i]==0: 
                TN += 1./sample_weight
            if ((y_predi[i]==0) and (y_truei[i]!=y_predi[i])): 
                FN += 1./sample_weight 
        f_beta_i = ((1+beta1**2)*TP)/((1+beta1**2)*TP + FP + (beta1**2)*FN)
        g_beta_i = (TP)/(TP+FP+beta2*FN)

        f_beta += f_beta_i
        g_beta += g_beta_i

    return {'F_beta_macro':f_beta/y_true.shape[1], 'G_beta_macro':g_beta/y_true.shape[1]}

def get_appropriate_bootstrap_samples(y_true, n_bootstraping_samples):
    samples=[]
    while True:
        ridxs = np.random.randint(0, len(y_true), len(y_true))
        if y_true[ridxs].sum(axis=0).min() != 0:
            samples.append(ridxs)
            if len(samples) == n_bootstraping_samples:
                break
    return samples

def find_optimal_cutoff_threshold(target, predicted):
    """ 
    Find the optimal probability cutoff point for a classification model related to event rate
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    return optimal_threshold

def find_optimal_cutoff_thresholds(y_true, y_pred):
	return [find_optimal_cutoff_threshold(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])]

def find_optimal_cutoff_threshold_for_Gbeta(target, predicted, n_thresholds=100):
    thresholds = np.linspace(0.00,1,n_thresholds)
    scores = [challenge_metrics(target, predicted>t, single=True)['G_beta_macro'] for t in thresholds]
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx]

def find_optimal_cutoff_thresholds_for_Gbeta(y_true, y_pred):
    print("optimize thresholds with respect to G_beta")
    return [find_optimal_cutoff_threshold_for_Gbeta(y_true[:,k][:,np.newaxis], y_pred[:,k][:,np.newaxis]) for k in tqdm(range(y_true.shape[1]))]

def apply_thresholds(preds, thresholds):
	"""
		apply class-wise thresholds to prediction score in order to get binary format.
		BUT: if no score is above threshold, pick maximum. This is needed due to metric issues.
	"""
	tmp = []
	for p in preds:
		tmp_p = (p > thresholds).astype(int)
		if np.sum(tmp_p) == 0:
			tmp_p[np.argmax(p)] = 1
		tmp.append(tmp_p)
	tmp = np.array(tmp)
	return tmp

# DATA PROCESSING STUFF

def load_dataset(path, sampling_rate, release=False):
    if path.split('/')[-2] == 'ptbxl':
        # load and convert annotation data
        Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
        Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

        # Load raw signal data
        X = load_raw_data_ptbxl(Y, sampling_rate, path)

    elif path.split('/')[-2] == 'ICBEB':
        # load and convert annotation data
        Y = pd.read_csv(path+'icbeb_database.csv', index_col='ecg_id')
        Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

        # Load raw signal data
        X = load_raw_data_icbeb(Y, sampling_rate, path)

    return X, Y


def load_raw_data_icbeb(df, sampling_rate, path):

    if sampling_rate == 100:
        if os.path.exists(path + 'raw100.npy'):
            data = np.load(path+'raw100.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path + 'records100/'+str(f)) for f in tqdm(df.index)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw100.npy', 'wb'), protocol=4)
    elif sampling_rate == 500:
        if os.path.exists(path + 'raw500.npy'):
            data = np.load(path+'raw500.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path + 'records500/'+str(f)) for f in tqdm(df.index)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw500.npy', 'wb'), protocol=4)
    return data

def load_raw_data_ptbxl(df, sampling_rate, path):
    if sampling_rate == 100:
        if os.path.exists(path + 'raw100.npy'):
            data = np.load(path+'raw100.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_lr)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw100.npy', 'wb'), protocol=4)
    elif sampling_rate == 500:
        if os.path.exists(path + 'raw500.npy'):
            data = np.load(path+'raw500.npy', allow_pickle=True)
        else:
            data = [wfdb.rdsamp(path+f) for f in tqdm(df.filename_hr)]
            data = np.array([signal for signal, meta in data])
            pickle.dump(data, open(path+'raw500.npy', 'wb'), protocol=4)
    return data

def compute_label_aggregations(df, folder, ctype):

    df['scp_codes_len'] = df.scp_codes.apply(lambda x: len(x))

    aggregation_df = pd.read_csv(folder+'scp_statements.csv', index_col=0)

    if ctype in ['diagnostic', 'subdiagnostic', 'superdiagnostic']:

        def aggregate_all_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    tmp.append(key)
            return list(set(tmp))

        def aggregate_subdiagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_subclass
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        def aggregate_diagnostic(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in diag_agg_df.index:
                    c = diag_agg_df.loc[key].diagnostic_class
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        diag_agg_df = aggregation_df[aggregation_df.diagnostic == 1.0]
        if ctype == 'diagnostic':
            df['diagnostic'] = df.scp_codes.apply(aggregate_all_diagnostic)
            df['diagnostic_len'] = df.diagnostic.apply(lambda x: len(x))
        elif ctype == 'subdiagnostic':
            df['subdiagnostic'] = df.scp_codes.apply(aggregate_subdiagnostic)
            df['subdiagnostic_len'] = df.subdiagnostic.apply(lambda x: len(x))
        elif ctype == 'superdiagnostic':
            df['superdiagnostic'] = df.scp_codes.apply(aggregate_diagnostic)
            df['superdiagnostic_len'] = df.superdiagnostic.apply(lambda x: len(x))
    elif ctype == 'form':
        form_agg_df = aggregation_df[aggregation_df.form == 1.0]

        def aggregate_form(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in form_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['form'] = df.scp_codes.apply(aggregate_form)
        df['form_len'] = df.form.apply(lambda x: len(x))
    elif ctype == 'rhythm':
        rhythm_agg_df = aggregation_df[aggregation_df.rhythm == 1.0]

        def aggregate_rhythm(y_dic):
            tmp = []
            for key in y_dic.keys():
                if key in rhythm_agg_df.index:
                    c = key
                    if str(c) != 'nan':
                        tmp.append(c)
            return list(set(tmp))

        df['rhythm'] = df.scp_codes.apply(aggregate_rhythm)
        df['rhythm_len'] = df.rhythm.apply(lambda x: len(x))
    elif ctype == 'all':
        df['all_scp'] = df.scp_codes.apply(lambda x: list(set(x.keys())))

    return df

def select_data(XX,YY, ctype, min_samples, outputfolder):
    # convert multilabel to multi-hot
    mlb = MultiLabelBinarizer()

    if ctype == 'diagnostic':
        X = XX[YY.diagnostic_len > 0]
        Y = YY[YY.diagnostic_len > 0]
        mlb.fit(Y.diagnostic.values)
        y = mlb.transform(Y.diagnostic.values)
    elif ctype == 'subdiagnostic':
        counts = pd.Series(np.concatenate(YY.subdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.subdiagnostic = YY.subdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['subdiagnostic_len'] = YY.subdiagnostic.apply(lambda x: len(x))
        X = XX[YY.subdiagnostic_len > 0]
        Y = YY[YY.subdiagnostic_len > 0]
        mlb.fit(Y.subdiagnostic.values)
        y = mlb.transform(Y.subdiagnostic.values)
    elif ctype == 'superdiagnostic':
        counts = pd.Series(np.concatenate(YY.superdiagnostic.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.superdiagnostic = YY.superdiagnostic.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['superdiagnostic_len'] = YY.superdiagnostic.apply(lambda x: len(x))
        X = XX[YY.superdiagnostic_len > 0]
        Y = YY[YY.superdiagnostic_len > 0]
        mlb.fit(Y.superdiagnostic.values)
        y = mlb.transform(Y.superdiagnostic.values)
    elif ctype == 'form':
        # filter
        counts = pd.Series(np.concatenate(YY.form.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.form = YY.form.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['form_len'] = YY.form.apply(lambda x: len(x))
        # select
        X = XX[YY.form_len > 0]
        Y = YY[YY.form_len > 0]
        mlb.fit(Y.form.values)
        y = mlb.transform(Y.form.values)
    elif ctype == 'rhythm':
        # filter 
        counts = pd.Series(np.concatenate(YY.rhythm.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.rhythm = YY.rhythm.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['rhythm_len'] = YY.rhythm.apply(lambda x: len(x))
        # select
        X = XX[YY.rhythm_len > 0]
        Y = YY[YY.rhythm_len > 0]
        mlb.fit(Y.rhythm.values)
        y = mlb.transform(Y.rhythm.values)
    elif ctype == 'all':
        # filter 
        counts = pd.Series(np.concatenate(YY.all_scp.values)).value_counts()
        counts = counts[counts > min_samples]
        YY.all_scp = YY.all_scp.apply(lambda x: list(set(x).intersection(set(counts.index.values))))
        YY['all_scp_len'] = YY.all_scp.apply(lambda x: len(x))
        # select
        X = XX[YY.all_scp_len > 0]
        Y = YY[YY.all_scp_len > 0]
        mlb.fit(Y.all_scp.values)
        y = mlb.transform(Y.all_scp.values)
    else:
        pass

    # save LabelBinarizer
    with open(outputfolder+'mlb.pkl', 'wb') as tokenizer:
        pickle.dump(mlb, tokenizer)

    return X, Y, y, mlb

def preprocess_signals(X_train, X_validation, X_test, outputfolder):
    # Standardize data such that mean 0 and variance 1
    ss = StandardScaler()
    ss.fit(np.vstack(X_train).flatten()[:,np.newaxis].astype(float))
    
    # Save Standardizer data
    with open(outputfolder+'standard_scaler.pkl', 'wb') as ss_file:
        pickle.dump(ss, ss_file)

    return apply_standardizer(X_train, ss), apply_standardizer(X_validation, ss), apply_standardizer(X_test, ss)

def apply_standardizer(X, ss):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(ss.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp


# DOCUMENTATION STUFF

def generate_ptbxl_summary_table(selection=None, folder='../output/'):

    exps = ['exp0', 'exp1', 'exp1.1', 'exp1.1.1', 'exp2', 'exp3']
    metric1 = 'macro_auc' 

    # get models
    models = {}
    for i, exp in enumerate(exps):
        if selection is None:
            exp_models = [m.split('/')[-1] for m in glob.glob(folder+str(exp)+'/models/*')]
        else:
            exp_models = selection
        if i == 0:
            models = set(exp_models)
        else:
            models = models.union(set(exp_models))

    results_dic = {'Method':[], 
                'exp0_AUC':[], 
                'exp1_AUC':[], 
                'exp1.1_AUC':[], 
                'exp1.1.1_AUC':[], 
                'exp2_AUC':[],
                'exp3_AUC':[]
                }

    for m in models:
        results_dic['Method'].append(m)
        
        for e in exps:
            
            try:
                me_res = pd.read_csv(folder+str(e)+'/models/'+str(m)+'/results/te_results.csv', index_col=0)
    
                mean1 = me_res.loc['point'][metric1]
                unc1 = max(me_res.loc['upper'][metric1]-me_res.loc['point'][metric1], me_res.loc['point'][metric1]-me_res.loc['lower'][metric1])

                results_dic[e+'_AUC'].append("%.3f(%.2d)" %(np.round(mean1,3), int(unc1*1000)))

            except FileNotFoundError:
                results_dic[e+'_AUC'].append("--")
            
            
    df = pd.DataFrame(results_dic)
    df_index = df[df.Method.isin(['naive', 'ensemble'])]
    df_rest = df[~df.Method.isin(['naive', 'ensemble'])]
    df = pd.concat([df_rest, df_index])
    df.to_csv(folder+'results_ptbxl.csv')

    titles = [
        '### 1. PTB-XL: all statements',
        '### 2. PTB-XL: diagnostic statements',
        '### 3. PTB-XL: Diagnostic subclasses',
        '### 4. PTB-XL: Diagnostic superclasses',
        '### 5. PTB-XL: Form statements',
        '### 6. PTB-XL: Rhythm statements'        
    ]

    # helper output function for markdown tables
    our_work = 'https://arxiv.org/abs/2004.13701'
    our_repo = 'https://github.com/helme/ecg_ptbxl_benchmarking/'
    md_source = ''
    for i, e in enumerate(exps):
        md_source += '\n '+titles[i]+' \n \n'
        md_source += '| Model | AUC &darr; | paper/source | code | \n'
        md_source += '|---:|:---|:---|:---| \n'
        for row in df_rest[['Method', e+'_AUC']].sort_values(e+'_AUC', ascending=False).values:
            md_source += '| ' + row[0].replace('fastai_', '') + ' | ' + row[1] + ' | [our work]('+our_work+') | [this repo]('+our_repo+')| \n'
    print(md_source)

def ICBEBE_table(selection=None, folder='../output/'):
    cols = ['macro_auc', 'F_beta_macro', 'G_beta_macro']

    if selection is None:
        models = [m.split('/')[-1].split('_pretrained')[0] for m in glob.glob(folder+'exp_ICBEB/models/*')]
    else:
        models = [] 
        for s in selection:
            #if s != 'Wavelet+NN':
                models.append(s)

    data = []
    for model in models:
        me_res = pd.read_csv(folder+'exp_ICBEB/models/'+model+'/results/te_results.csv', index_col=0)
        mcol=[]
        for col in cols:
            mean = me_res.ix['point'][col]
            unc = max(me_res.ix['upper'][col]-me_res.ix['point'][col], me_res.ix['point'][col]-me_res.ix['lower'][col])
            mcol.append("%.3f(%.2d)" %(np.round(mean,3), int(unc*1000)))
        data.append(mcol)
    data = np.array(data)

    df = pd.DataFrame(data, columns=cols, index=models)
    df.to_csv(folder+'results_icbeb.csv')

    df_rest = df[~df.index.isin(['naive', 'ensemble'])]
    df_rest = df_rest.sort_values('macro_auc', ascending=False)
    our_work = 'https://arxiv.org/abs/2004.13701'
    our_repo = 'https://github.com/helme/ecg_ptbxl_benchmarking/'

    md_source = '| Model | AUC &darr; |  F_beta=2 | G_beta=2 | paper/source | code | \n'
    md_source += '|---:|:---|:---|:---|:---|:---| \n'
    for i, row in enumerate(df_rest[cols].values):
        md_source += '| ' + df_rest.index[i].replace('fastai_', '') + ' | ' + row[0] + ' | ' + row[1] + ' | ' + row[2] + ' | [our work]('+our_work+') | [this repo]('+our_repo+')| \n'
    print(md_source)
    

In [ ]:
from utils import utils

sampling_frequency=100
datafolder= '/gdrive/My Drive/ICBEB/'
task='all'
outputfolder='../output0/'

# Load data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

num_classes = 9         # <=== number of classes in the finetuning dataset
input_shape = [1000,12] # <=== shape of samples, [None, 12] in case of different lengths

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((6187,), (6187, 9), (690,), (690, 9))

In [ ]:
X_tr = []
for i in range(len(X_train)):
    x = []
    for j in range(12):
        p = X_train[i][:1000,j]
        if p.shape[0]!= 1000:
            d = abs(p.shape[0]-1000)//2
            p = np.pad(p,(d,d))
        x.append(p)
    X_tr.append(np.transpose(x))
X_tr = np.array(X_tr)
X_tr.shape

(6187, 1000, 12)

In [ ]:
X_te = []
for i in range(len(X_val)):
    x = []
    for j in range(12):
        p = X_val[i][:1000,j]
        if p.shape[0]!= 1000:
            d = abs(p.shape[0]-1000)//2
            p = np.pad(p,(d,d))
        x.append(p)
    X_te.append(np.transpose(x))
X_te = np.array(X_te)
X_te.shape

(690, 1000, 12)

In [ ]:
cd /gdrive/MyDrive/

/gdrive/MyDrive


Model Definition:

In [ ]:
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model
import numpy as np


class ResidualUnit(object):
    """Residual unit block (unidimensional).
    Parameters
    ----------
    n_samples_out: int
        Number of output samples.
    n_filters_out: int
        Number of output filters.
    kernel_initializer: str, optional
        Initializer for the weights matrices. See Keras initializers. By default it uses
        'he_normal'.
    dropout_keep_prob: float [0, 1), optional
        Dropout rate used in all Dropout layers. Default is 0.8
    kernel_size: int, optional
        Kernel size for convolutional layers. Default is 17.
    preactivation: bool, optional
        When preactivation is true use full preactivation architecture proposed
        in [1]. Otherwise, use architecture proposed in the original ResNet
        paper [2]. By default it is true.
    postactivation_bn: bool, optional
        Defines if you use batch normalization before or after the activation layer (there
        seems to be some advantages in some cases:
        https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md).
        If true, the batch normalization is used before the activation
        function, otherwise the activation comes first, as it is usually done.
        By default it is false.
    activation_function: string, optional
        Keras activation function to be used. By default 'relu'.
    References
    ----------
    .. [1] K. He, X. Zhang, S. Ren, and J. Sun, "Identity Mappings in Deep Residual Networks,"
           arXiv:1603.05027 [cs], Mar. 2016. https://arxiv.org/pdf/1603.05027.pdf.
    .. [2] K. He, X. Zhang, S. Ren, and J. Sun, "Deep Residual Learning for Image Recognition," in 2016 IEEE Conference
           on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 770-778. https://arxiv.org/pdf/1512.03385.pdf
    """

    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1]
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2]
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]


def get_model(n_classes, last_layer='sigmoid'):
    kernel_size = 16
    kernel_initializer = 'he_normal'
    signal = Input(shape=(1000, 12), dtype=np.float32, name='signal')
    x = signal
    x = Conv1D(8, kernel_size, padding='same', use_bias=False,
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x, y = ResidualUnit(128, 8, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, x])
    x, y = ResidualUnit(64, 16, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, y = ResidualUnit(32, 32, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, _ = ResidualUnit(16, 64, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x = Flatten()(x)
    diagn = Dense(9, activation=last_layer, kernel_initializer=kernel_initializer)(x)
    model = Model(signal, diagn)
    return model


if __name__ == "__main__":
    model = get_model(6)
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
signal (InputLayer)             [(None, 1000, 12)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1000, 8)      1536        signal[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1000, 8)      32          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 1000, 8)      0           batch_normalization[0][0]        
______________________________________________________________________________________________

Train:

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, fbeta_score, roc_auc_score, roc_curve, roc_curve, auc , f1_score
# from model import get_model
import argparse
# from datasets import ECGSequence
import tensorflow as tf
import math
import mlflow
import mlflow.tensorflow

In [ ]:
mlflow.tensorflow.autolog()

In [ ]:
with mlflow.start_run() as run:

    def find_optimal_cutoff_threshold(target, predicted):
        """ 
        Find the optimal probability cutoff point for a classification model related to event rate
        """
        fpr, tpr, threshold = roc_curve(target, predicted)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = threshold[optimal_idx]
        return optimal_threshold

    def find_optimal_cutoff_thresholds(y_true, y_pred):
	    return [find_optimal_cutoff_threshold(y_true[:,i], y_pred[:,i]) for i in range(y_true.shape[1])]

    def apply_thresholds(preds, thresholds):
        """
            apply class-wise thresholds to prediction score in order to get binary format.
            BUT: if no score is above threshold, pick maximum. This is needed due to metric issues.
        """
        tmp = []
        for p in preds:
            tmp_p = (p > thresholds).astype(int)
            if np.sum(tmp_p) == 0:
                tmp_p[np.argmax(p)] = 1
            tmp.append(tmp_p)
        tmp = np.array(tmp)
        return tmp

    def step_decay(epoch):
        initial_lrate = 0.001
        drop = 0.4
        epochs_drop = 20.0
        lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        return lrate

    # lscheduler = tf.keras.callbacks.LearningRateScheduler(step_decay)
    lr = 0.001
    callbacks = [ReduceLROnPlateau(monitor='val_loss',
                                    factor=0.1,
                                    patience=7,
                                    min_lr=lr / 100),
                    EarlyStopping(patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                                min_delta=0.00001)]

    adam = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['AUC'])

    num_folds = 5
    epochs = 20
    batch_size= 32
    kfold = KFold(num_folds)

    histories = []
    fold_no = 1
    for train, test in kfold.split(X_tr, y_train):

        h = model.fit(X_tr[train], y_train[train],
                        epochs=epochs, batch_size=batch_size,
                        validation_data=(X_tr[test],y_train[test]),
                        callbacks=callbacks)

        print("iteration ", str(fold_no))
        histories.append(h)
        fold_no += 1

    # predictions = model.predict(X_te)
    test_auc = model.evaluate(X_te, y_val)[1]
    print("Test auc avg: ", test_auc)

    y_pred = model.predict(X_te)
    thresholds =  find_optimal_cutoff_thresholds_for_Gbeta(y_val , y_pred)
    y_pred_binary = apply_thresholds(y_pred, thresholds)

    f1 = f1_score(y_val, y_pred_binary, average='macro')

    mlflow.log_metric("Test AUC", test_auc) 
    mlflow.log_metric('macro_f1_score' , f1) 

    mlflow.keras.log_model(model, "nature_model")
                        #    custom_objects=
                        #    {"log_cosh_dice_loss": log_cosh_dice_loss,"dice_coef":dice_coef})

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/20
155/155 [==============================] - 27s 146ms/step - loss: 0.3328 - auc: 0.7320 - val_loss: 0.2993 - val_auc: 0.8130
Epoch 2/20
155/155 [==============================] - 21s 135ms/step - loss: 0.2576 - auc: 0.8509 - val_loss: 0.2576 - val_auc: 0.8654
Epoch 3/20
155/155 [==============================] - 20s 132ms/step - loss: 0.2303 - auc: 0.8868 - val_loss: 0.2753 - val_auc: 0.8674
Epoch 4/20
155/155 [==============================] - 21s 134ms/step - loss: 0.2094 - auc: 0.9086 - val_loss: 0.2770 - val_auc: 0.8685
Epoch 5/20
155/155 [==============================] - 21s 133ms/step - loss: 0.1892 - auc: 0.9276 - val_loss: 0.1977 - val_auc: 0.9294
Epoch 6/20
155/155 [==============================] - 21s 137ms/step - loss: 0.1770 - auc: 0.9382 - val_loss: 0.1734 - val_auc: 0.9426
Epoch 7/20
155/155 [==============================] - 21s 135ms/step - loss: 0.1616 - auc: 0.9488 - val_loss: 0.1745 - val_auc: 0.9442
Epoch 8/20
155/155 [==============================] - 2

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmp2bpv2tli/model/data/model/assets
iteration  1


2021/08/10 04:53:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.7/dist-packages/mlflow/tensorflow.py:781: UserWarning: Logging to MLflow failed: Changing param values is not allowed. Param with key='opt_learning_rate' was already logged with value='0.001' for run ID='21e68a9132b0469e940144868621bb09'. Attempted logging new value '0.000100000005'."


...

INFO:tensorflow:Assets written to: /tmp/tmpryrtlgr6/model/data/model/assets
iteration  5
22/22 [==============================] - 1s 29ms/step - loss: 0.1579 - auc: 0.9571
Test auc avg:  0.9571318626403809


  0%|          | 0/9 [00:00<?, ?it/s]

optimize thresholds with respect to G_beta


100%|██████████| 9/9 [00:08<00:00,  1.08it/s]
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmplx__v1o6/model/data/model/assets


In [ ]:
 model.evaluate(X_te, y_val)[1]

22/22 [==============================] - 1s 30ms/step - loss: 0.1579 - auc: 0.9571


0.9571318626403809

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 746 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.6-py3-none-any.whl size=19263 sha256=e4f925a541939c8d8a08ba4c69f19d42516f01b8be93ce13026378ff88d12a11
  Stored in directory: /root/.cache/pip/wheels/d5/8c/c4/8d9cbca4fa19bf64887b4a91914194bb9033f1a7cbb344d5ab
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok

get_ipython().system_raw("mlflow ui --port 5000 &")

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "1w9tvYSl88onvrzvZJMxMcDDb9Y_3NHGDZssJWJ6rdcw9TsBN"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://3780d198fd00.ngrok.io


In [ ]:
from pyngrok import ngrok

get_ipython().system_raw("mlflow ui --port 5000 &")

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "1w9tvYSl88onvrzvZJMxMcDDb9Y_3NHGDZssJWJ6rdcw9TsBN"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://a3de6dc54841.ngrok.io
